In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import find_ball_initial_velocity, PITCH_START_POSITION
from iiwa_batter.save_load import load_control_vector_trajectory, load_dataviewer_trajectory

from iiwa_batter.swing_simulator import setup_simulator, run_swing_simulation, reset_systems

In [ ]:
meshcat = StartMeshcat()

In [ ]:
robot_constraints = JOINT_CONSTRAINTS["iiwa14"]

simulator, diagram = setup_simulator(torque_trajectory={0:np.ones(NUM_JOINTS)*40}, dt=PITCH_DT, add_contact=False, robot_constraints=robot_constraints, meshcat=meshcat)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=4.0,
    initial_joint_positions=np.array([0] * NUM_JOINTS),
    initial_joint_velocities=np.array([0] * NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
    record_state=True,
)

In [ ]:
#setup_simulator(dt=CONTACT_DT, meshcat=None)

In [ ]:
torque_trajectory={
    0: np.ones(NUM_JOINTS)*-40,
    1: np.ones(NUM_JOINTS)*40
}

simulator, diagram = setup_simulator(torque_trajectory, dt=PITCH_DT, add_contact=True, meshcat=meshcat)

from iiwa_batter.swing_optimization.stochastic_gradient_descent import find_initial_positions

initial_joint_positions = find_initial_positions(simulator, diagram, robot_constraints, 1)

reset_systems(diagram)

run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=2*CONTACT_DT,
    initial_joint_positions=np.array([0] * NUM_JOINTS),
    initial_joint_velocities=np.array([0] * NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
)
